In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics,preprocessing,svm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from imblearn.under_sampling import RandomUnderSampler
from pydotplus import graph_from_dot_data
from IPython.display import Image
from keras import layers, models
from keras.optimizer_v2 import adam
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence, text
from keras.utils import vis_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
%matplotlib inline

dataset citation:
https://research.aalto.fi/en/datasets/phishstorm-phishing-legitimate-url-dataset
"
Description
URLs dataset with features built and used for evaluation in the paper "PhishStorm: Detecting Phishing with Streaming Analytics" published in IEEE TNSM.
The dataset contains 96,018 URLs: 48,009 legitimate URLs and 48,009 phishing URLs.

This is a CSV file where the "domain" column provides a unique identifier for each entry (which is actually a URL). The "label" column provides the domain entry status, 0: legitimate / 1:phishing.
Other columns provide computed values for features introduced in [1].

Please refer to the following publication when citing this dataset:
[1] S. Marchal, J. Francois, R. State, and T. Engel. PhishStorm: Detecting Phishing with Streaming Analytics. IEEE Transactions on Network and Service Management (TNSM), 11(4):458-471, 2014.
"

In [ ]:
seed = 69 # fix random seed
df = pd.read_csv('urlset.csv',
                 encoding='UTF-8',
                 encoding_errors='ignore',
                 on_bad_lines='skip',
                 low_memory=False)

In [ ]:
df.head(5)

In [ ]:
df.shape

### there are some unreadable row, we have to skip therefore the dataset contains total 96003 samples. However, there is two duplicates values, we have to drop them. Therefore, the dataset contains total 96001 samples

In [ ]:
# remove other unneccessary columns, only keep url and label and make sure url type is string
df = df[['domain','label']].astype({'domain':str,'label':float})
df = df.drop_duplicates(ignore_index=False)

In [ ]:
df.head(5)

In [ ]:
all_char = list(''.join(df.domain.values.tolist()))
unique_char = np.unique(all_char,return_counts=True)
print("Raw data total {} unique characters:".format(unique_char[0].shape[0]))
print(unique_char[0])

##### preprocessing data
1. fit character into numerical values
2. only keep the url length up to 75 characters
3. if url does not have 75 characters then padding in front with 0
4. remove all $\pm\infty$ and nan values
5. balance binary dataset
-------------------------------------
6. use PCA to reduce dimension

In [ ]:
char_tok = text.Tokenizer(char_level=True) # character function
char_tok.fit_on_texts(df.domain) # fit character
data = char_tok.texts_to_sequences(df.domain)
num_char = len(char_tok.word_index)+1
max_length = max([len(n) for n in data])
length = 75
epochs = 10
batch_size = 100
print("Total {} possible length of characters".format(max_length)) # check maximum possible length
print('='*80)
data = sequence.pad_sequences(data,maxlen=length,dtype=np.float64) 
print("Total {} samples and only use {} features".format(*data.shape))
print('='*80)
# save data and its target to a new dataframe
data = pd.DataFrame(np.hstack((data,df[['label']].values)))
# drop nan, inf, or -inf values
data = data.replace([np.inf, -np.inf], np.nan).dropna()
# use PCA to reduce dimension of feature
columns = data.columns
rus = RandomUnderSampler(random_state=seed) # random balance smaple function
standard = preprocessing.StandardScaler() # standardize data function
X, y = rus.fit_resample(data[columns[:-1]],data[columns[-1]])
print("After balance sampling:")
print(f"|phishing:  {len(y[y==1])}  | non-phishing: {len(y[y==0])}")
print('='*80)
# split dataset for training, validation and testing
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=seed,stratify=y)
print("split dataset into training and testing:")
print(f"Training size: {y_train.shape[0]}; Testing size: {y_test.shape[0]}")
print('='*80)
del df

In [ ]:
def CNN_model(num_char=373,length=75):
    '''
    convolutional neural network
    '''
    model = models.Sequential([Embedding(num_char,128,input_length=length), # embedding
                               layers.Dropout(0.5), 
                               layers.Conv1D(256, 3), # conv layer 1
                               layers.ELU(), # Exponential Linear Unit
                               layers.BatchNormalization(), # batach normalization 
                               layers.Dropout(0.5),
                               layers.Conv1D(512, 6), # conv layer 2
                               layers.ELU(), # Exponential Linear Unit
                               layers.BatchNormalization(), # batach normalization 
                               layers.Dropout(0.5),
                               layers.Conv1D(1024,8), # conv layer 3
                               layers.ELU(), # Exponential Linear Unit
                               layers.BatchNormalization(), # batach normalization 
                               layers.Dropout(0.5),
                               layers.Flatten(),
                               layers.Dense(1024,activation='relu'), # hidden layer 1
                               layers.BatchNormalization(), # batach normalization 
                               layers.Dropout(0.5),
                               layers.Dense(256, activation='relu'), # hidden layer 2
                               layers.BatchNormalization(), # batach normalization 
                               layers.Dropout(0.5),
                               layers.Dense(64,  activation='relu'), # hidden layer 3
                               layers.BatchNormalization(), # batach normalization 
                               layers.Dropout(0.5),
                               layers.Dense(1,activation='sigmoid')]) # output layer
    model_compile(model)
    return model

def LSTM_model(num_char=373,length=75):
    '''
    model with one embedding layer,
    follow up with 25% drop out and batach normalization
    lstm layer with 128 units and 50% recurrent drop out
    '''
    model =models.Sequential([Embedding(num_char,128,input_length=length), # embedding
                              layers.Dropout(0.25),
                              layers.BatchNormalization(), # batach normalization 
                              layers.LSTM(128,recurrent_dropout=0.5), # long-short term memory
                              layers.Dropout(0.5),
                              layers.Flatten(),
                              layers.Dense(1,activation='sigmoid')]) # output layer
    model_compile(model)
    return model

def LSTM_larger_model(num_char=373,length=75):
    '''
    model with one embedding layer,
    follow up with 25% drop out and batach normalization
    lstm layer with 256 units and 50% recurrent drop out
    lstm layer with 512 units and 50% recurrent drop out
    '''
    model =models.Sequential([Embedding(num_char,128,input_length=length), # embedding
                              layers.Dropout(0.25),
                              layers.BatchNormalization(), # batach normalization 
                              layers.LSTM(256,recurrent_dropout=0.5,
                                          return_sequences=True,
                                          input_shape=(75,128)), # long-short term memory
                              layers.Dropout(0.5),
                              layers.BatchNormalization(), # batach normalization
                              layers.LSTM(512,recurrent_dropout=0.5), # long-short term memory
                              layers.Dropout(0.5),
                              layers.Flatten(),
                              layers.Dense(1,activation='sigmoid')]) # output layer
    model_compile(model)
    return model
    

def model_compile(model):
    '''
    model compile with binary corss entropy, adam optimizer with learning rate 0.005
    '''
    model.compile(loss='binary_crossentropy',
                  optimizer=adam.Adam(learning_rate=0.005),
                  metrics=['accuracy'])

def train_model(model,X_train,y_train,epochs=10,batch_size=32,filename='cnn'):
#     print("model summary")
#     print(model.summary())
    print(f"training model with {epochs} epochs and batches of size {batch_size}:")
    earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
    mcp_save = ModelCheckpoint(filename+'.hdf5', save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7,
                                       verbose=1,min_delta=1e-4, mode='min')
    return model.fit(X_train,y_train,validation_split=0.2,epochs=epochs,
                     batch_size=batch_size,use_multiprocessing=True,
                     callbacks=[earlyStopping, mcp_save,reduce_lr_loss])

def load_best_model(model,filename='cnn'):
    return model.load_weights(filepath = filename+'.hdf5')


def evaluate_model(model,X_test,y_test):
    print("model summary")
    print(model.summary())
    print(f"testing model:")
    result = model.evaluate(X_test,y_test)
    print("Model score: {:.04f} and accuracy: {:0.2f}%".format(result[0],result[1]*100))
    
def classification_report(model,X_test,y_test):
    print("claissification report:")
    print(metrics.classification_report(y_test,
                                  np.where(model.predict(X_test,
                                                         use_multiprocessing=True)>0.5,
                                           1,0)))
    
def export_plot(model,fname='cnn'):
    '''
    plot modell's structure
    '''
    vis_utils.plot_model(model,to_file=fname+'.png',show_shapes=True,
                         show_layer_names=True,expand_nested=True,dpi=96,)
    
def plot_acc(ax,hist,title):
    val = title if title == 'Accuracy' else 'Loss'
    ax.plot(hist.history[val.lower()],lw=2)
    ax.plot(hist.history['val_'+val.lower()],lw=2)
    ax.set(title=title,ylabel=val,xlabel='Epoch')
    ax.legend(['Train','Test'],fontsize='xx-large')
    ax.grid(True)

def false_negative_rate(cm):
    '''
    compute false negative rate
    '''
    tn, fp, fn, tp = cm.ravel()
    return fn/(fn+tp)

In [ ]:
cnn_model = CNN_model(num_char,length)
export_plot(cnn_model,'cnn')

In [ ]:
cnn_hist = train_model(cnn_model,X_train,y_train,filename='cnn')

In [ ]:
load_best_model(cnn_model,filename='cnn')
plt.rcParams.update({'font.size':15})
fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(20,8))
ax = ax.flatten()
plot_acc(ax[0],cnn_hist,'Accuracy')
plot_acc(ax[1],cnn_hist,'Loss')

In [ ]:
evaluate_model(cnn_model,X_test,y_test)

In [ ]:
plt.rcParams.update({'font.size':15})
ax = plt.figure(figsize=(8,8))
y_pred = np.where(cnn_model.predict(X_test,use_multiprocessing=True)>0.5,1,0)
cm = metrics.confusion_matrix(y_test,y_pred)
print("false negative rate: {:0.4f}%".format(false_negative_rate(cm)*100))
print('='*80)
sns.heatmap(cm,annot=True,fmt=".0f",linewidths=.5,square=True,cmap='rainbow_r')
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
plt.title('Confusion matrix\nTesting accuracy score:'
          '{:0.2f}%'.format(metrics.accuracy_score(y_test,y_pred)*100),size=18)
plt.show()

In [ ]:
classification_report(cnn_model,X_test,y_test)

In [ ]:
lstm_model = LSTM_model(num_char,length)
export_plot(lstm_model,'lstm')

In [ ]:
lstm_hist = train_model(lstm_model,X_train,y_train,filename='lstm1')

In [ ]:
load_best_model(lstm_model,filename='lstm1')
plt.rcParams.update({'font.size':15})
fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(20,8))
ax = ax.flatten()
plot_acc(ax[0],lstm_hist,'Accuracy')
plot_acc(ax[1],lstm_hist,'Loss')

In [ ]:
evaluate_model(lstm_model,X_test,y_test)

In [ ]:
plt.rcParams.update({'font.size':15})
ax = plt.figure(figsize=(8,8))
y_pred = np.where(lstm_model.predict(X_test,use_multiprocessing=True)>0.5,1,0)
cm = metrics.confusion_matrix(y_test,y_pred)
print("false negative rate: {:0.4f}%".format(false_negative_rate(cm)*100))
print('='*80)
sns.heatmap(cm,annot=True,fmt=".0f",linewidths=.5,square=True,cmap='rainbow_r')
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
plt.title('Confusion matrix\nTesting accuracy score:'
          '{:0.2f}%'.format(metrics.accuracy_score(y_test,y_pred)*100),size=18)
plt.show()

In [ ]:
classification_report(lstm_model,X_test,y_test)

In [ ]:
larger_lstm = LSTM_larger_model(num_char,length)
export_plot(larger_lstm,'larger_lstm')

In [ ]:
larger_lstm_hist = train_model(larger_lstm,X_train,y_train,filename='lstm2')

In [ ]:
load_best_model(larger_lstm,filename='lstm2')
plt.rcParams.update({'font.size':15})
fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(20,8))
ax = ax.flatten()
plot_acc(ax[0],larger_lstm_hist,'Accuracy')
plot_acc(ax[1],larger_lstm_hist,'Loss')

In [ ]:
evaluate_model(larger_lstm,X_test,y_test)

In [ ]:
plt.rcParams.update({'font.size':15})
ax = plt.figure(figsize=(8,8))
y_pred = np.where(larger_lstm.predict(X_test,use_multiprocessing=True)>0.5,1,0)
cm = metrics.confusion_matrix(y_test,y_pred)
print("false negative rate: {:0.4f}%".format(false_negative_rate(cm)*100))
print('='*80)
sns.heatmap(cm,annot=True,fmt=".0f",linewidths=.5,square=True,cmap='rainbow_r')
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
plt.title('Confusion matrix\nTesting accuracy score:'
          '{:0.2f}%'.format(metrics.accuracy_score(y_test,y_pred)*100),size=18)
plt.show()

In [ ]:
classification_report(larger_lstm,X_test,y_test)

#### Logistic regression without feature reduction

In [ ]:
print("Logistic regression without feature reduction:")
logit = LogisticRegression(penalty='l2',tol=0.0001,C=1,random_state=seed,
                           solver='liblinear',multi_class='ovr',max_iter=100,verbose=0)
logit.fit(X_train,y_train)
test_score = np.round(logit.score(X_test,y_test)*100,2)
print('='*80)
print(f"|+| test accuracy: \t{test_score}%")
print('='*80)
cm = metrics.confusion_matrix(y_test,logit.predict(X_test))
print("false negative rate: {:0.4f}%".format(false_negative_rate(cm)*100))
print('='*80)
plt.rcParams.update({'font.size':15})
ax = plt.figure(figsize=(8,8))
sns.heatmap(cm,annot=True,fmt=".0f",linewidths=.5,square=True,cmap='rainbow_r')
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
plt.title('Confusion matrix\nTesting accuracy score:'
          '{:0.2f}%'.format(test_score),size=18)
plt.show()

In [ ]:
'''
Modified from: Selecting dimensionality reduction with Pipeline and GridSearchCV
orginal authors: Robert McGibbon, Joel Nothman, Guillaume Lemaitre
'''
pipe = Pipeline([# the reduce_dim stage is populated by the param_grid
                 ("reduce_dim", "passthrough"),
                 ("classify", svm.LinearSVC(dual=False, max_iter=10000)),
                ])

N_FEATURES_OPTIONS = [15,30,45,60] # reduction features
C_OPTIONS = [1,10]
param_grid = [{"reduce_dim": [PCA(iterated_power=7,random_state=seed)],
               "reduce_dim__n_components": N_FEATURES_OPTIONS,
               "classify__C": C_OPTIONS,}]
reducer_labels = ["PCA"]

grid = GridSearchCV(pipe,n_jobs=1,param_grid=param_grid)
grid.fit(X, y)

mean_scores = np.array(grid.cv_results_["mean_test_score"])
# scores are in the order of param_grid iteration, which is alphabetical
mean_scores = mean_scores.reshape(len(C_OPTIONS), -1, len(N_FEATURES_OPTIONS))
# select score for best C
mean_scores = mean_scores.max(axis=0)
bar_offsets = np.arange(len(N_FEATURES_OPTIONS)) * (len(reducer_labels) + 1) + 0.5

plt.figure()
COLORS = "bgrcmyk"
for i, (label, reducer_scores) in enumerate(zip(reducer_labels, mean_scores)):
    plt.bar(bar_offsets+i,reducer_scores,label=label,color=COLORS[i])

plt.title("Comparing feature reduction techniques")
plt.xlabel("Reduced number of features")
plt.xticks(bar_offsets, N_FEATURES_OPTIONS)
plt.ylabel("classification accuracy")
plt.ylim((0, 1))
plt.legend(loc="upper left")
plt.show()
# only use 30 features from pca
pca = PCA(n_components=15,iterated_power=7,random_state=seed)
new_X = pca.fit_transform(X)

# standardize data
new_X = standard.fit_transform(new_X) # normalize all features
std_X = new_X[X_train.index.values,:] # allocate X train, sample 
std_y = y.values[y_train.index.values] # allocate train target
std_X_test = new_X[X_test.index.values,:] # allocate X test 
std_y_test = y.values[y_test.index.values] # allocate test target

In [ ]:
logit = LogisticRegression(penalty='l2',tol=0.0001,C=1,random_state=seed,
                           solver='liblinear',multi_class='ovr',max_iter=100,verbose=1)
logit.fit(std_X,std_y)
test_score = np.round(logit.score(std_X_test,std_y_test)*100,2)
print('='*80)
print(f"|+| test accuracy: \t{test_score}%")
print('='*80)
cm = metrics.confusion_matrix(std_y_test,logit.predict(std_X_test))
print("false negative rate: {:0.4f}%".format(false_negative_rate(cm)*100))
print('='*80)
plt.rcParams.update({'font.size':15})
ax = plt.figure(figsize=(8,8))
sns.heatmap(cm,annot=True,fmt=".0f",linewidths=.5,square=True,cmap='rainbow_r')
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
plt.title('Confusion matrix\nTesting accuracy score:{}%'.format(test_score),size=18)
plt.show()

In [ ]:
precision, recall, _ = metrics.precision_recall_curve(std_y_test,logit.predict(std_X_test))
disp = metrics.PrecisionRecallDisplay(precision=precision, recall=recall)
disp.plot()
disp.ax_.set(title="Precision-Recall curve")
disp.ax_.grid(True)
plt.show()

In [ ]:
C = 1 # SVM regularization parameter
max_iter = 100000
svm_models = [svm.LinearSVC(C=C,max_iter=max_iter),
              svm.SVC(kernel='rbf',gamma='auto',max_iter=max_iter,C=C)]
svm_models = [clf.fit(std_X,std_y) for clf in svm_models]

In [ ]:
svm_titles = ['Linear SVC', 'SVC with RBF kernel']
for i, clf in enumerate(svm_models):
    test_score = np.round(clf.score(std_X_test,std_y_test)*100,2)
    print("="*80)
    print(f"||{svm_titles[i]}\ttesting score: {test_score}")
    print('='*80)
    print(f"|+| test accuracy: \t{test_score}%")
    print('='*80)
    cm = metrics.confusion_matrix(std_y_test,clf.predict(std_X_test))
    print("false negative rate: {:0.4f}%".format(false_negative_rate(cm)*100))
    print('='*80)
    plt.rcParams.update({'font.size':15})
    ax = plt.figure(figsize=(8,8))
    sns.heatmap(cm,annot=True,fmt=".0f",linewidths=.5,square=True,cmap='rainbow_r')
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')
    plt.title('Confusion matrix\nTesting accuracy score:{}%'.format(test_score),size=18)
    plt.show()

In [ ]:
tree_models = []
accuracies = []
criterion = ['gini','entropy']
depth = 3
for j, c in enumerate(criterion):
    tree_models.append(DecisionTreeClassifier(criterion=c,max_depth=depth,random_state=seed))
    tree_models[j].fit(std_X,std_y)
    accuracies.append(np.round(tree_models[j].score(std_X_test,std_y_test)*100,2))
    print('='*80)
    print(f"{c} impurity max tree depth of {depth} accuracy: {accuracies[j]}%")
    print('='*80)
    cm = metrics.confusion_matrix(std_y_test,tree_models[j].predict(std_X_test))
    print("false negative rate: {:0.4f}%".format(false_negative_rate(cm)*100))
    print('='*80)
    plt.rcParams.update({'font.size':15})
    ax = plt.figure(figsize=(8,8))
    sns.heatmap(cm,annot=True,fmt=".0f",linewidths=.5,square=True,cmap='rainbow_r')
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')
    plt.title('Confusion matrix\nTesting accuracy score:{}%'.format(accuracies[j]),size=18)
    plt.show()

In [ ]:
print(f"The highest accuracy with depth {depth}")
dot_data = export_graphviz(tree_models[0],filled=True,rounded=True,
                           class_names=['phishing','non-phishing'],
                           feature_names=[f'ch{i+1}' for i in range(std_X.shape[1])],out_file=None)
dot_graph = graph_from_dot_data(dot_data)
dot_graph.write_png('gini0.png')
Image('gini0.png',height=1000,width=1000)

In [ ]:
tree_models = []
accuracies = []
criterion = ['gini','entropy']
depth = 6
for j, c in enumerate(criterion):
    tree_models.append(DecisionTreeClassifier(criterion=c,max_depth=depth,random_state=seed))
    tree_models[j].fit(std_X,std_y)
    accuracies.append(np.round(tree_models[j].score(std_X_test,std_y_test)*100,2))
    print('='*80)
    print(f"{c} impurity max tree depth of {depth} accuracy: {accuracies[j]}%")
    print('='*80)
    cm = metrics.confusion_matrix(std_y_test,tree_models[j].predict(std_X_test))
    print("false negative rate: {:0.4f}%".format(false_negative_rate(cm)*100))
    print('='*80)
    plt.rcParams.update({'font.size':15})
    ax = plt.figure(figsize=(8,8))
    sns.heatmap(cm,annot=True,fmt=".0f",linewidths=.5,square=True,cmap='rainbow_r')
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')
    plt.title('Confusion matrix\nTesting accuracy score:{}%'.format(accuracies[j]),size=18)
    plt.show()

In [ ]:
print(f"The highest accuracy with depth {depth}")
dot_data = export_graphviz(tree_models[0],filled=True,rounded=True,
                           class_names=['phishing','non-phishing'],
                           feature_names=[f'ch{i+1}' for i in range(std_X.shape[1])],out_file=None)
dot_graph = graph_from_dot_data(dot_data)
dot_graph.write_png('gini1.png')
Image('gini1.png',height=1000,width=1000)

In [ ]:
adas = []
for j in range(2):
    adas.append(AdaBoostClassifier(base_estimator=tree_models[j],n_estimators=500,
                                   learning_rate=0.1,random_state=seed))
    adas[j].fit(std_X,std_y)
    test_score = np.round(adas[j].score(std_X_test,std_y_test)*100,2)
    print("="*80)
    print(f"Adaboost with decision tree base {criterion[j]} of depth {depth} accuracy:"
          f"{test_score}%")
    print("="*80)
    cm = metrics.confusion_matrix(std_y_test,adas[j].predict(std_X_test))
    print("false negative rate: {:0.4f}%".format(false_negative_rate(cm)*100))
    print('='*80)
    plt.rcParams.update({'font.size':15})
    ax = plt.figure(figsize=(8,8))
    sns.heatmap(cm,annot=True,fmt=".0f",linewidths=.5,square=True,cmap='rainbow_r')
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')
    plt.title('Confusion matrix\nTesting accuracy score:{}%'.format(test_score),size=18)
    plt.show()